## Bronze Layer

#### Criei um volume delta já que no free tier o acesso a mnt/ é negado

#### Verificando os dados de teste disponíveis

In [0]:
%py
# Definir caminhos base
# BRONZE_PATH = "/mnt/delta/bronze"
SOURCE_PATH = "dbfs:/databricks-datasets"
BRONZE_PATH = "/Volumes/workspace/default/delta/bronze/retail"
SILVER_PATH = "/Volumes/workspace/default/delta/silver/retail"
GOLD_PATH = "/Volumes/workspace/default/delta/gold/retail"

# Criando diretório Silver
# Definir caminhos base

# Criar diretórios se não existirem
dbutils.fs.mkdirs(BRONZE_PATH)
dbutils.fs.mkdirs(SILVER_PATH)
dbutils.fs.mkdirs(GOLD_PATH)

### Dataset de retails


In [0]:
%fs
ls /databricks-datasets/retail-org

### Carregando os arquivos

#### active_promotions

In [0]:
dbutils.fs.ls("dbfs:/databricks-datasets/retail-org/purchase_orders/")

In [0]:
fold = "dbfs:/databricks-datasets/retail-org/purchase_orders/purchase_orders.xml"
# Lê o arquivo como texto para inspecionar o conteúdo bruto
raw_xml = spark.read.text(fold)
# Mostra algumas linhas
raw_xml.show(20, truncate=False)

In [0]:
fold = "dbfs:/databricks-datasets/retail-org/purchase_orders/purchase_orders.xml"

purchase_items = (
    spark.read.format("xml")
    .option("rowTag", "purchase_item")  # 🔹 cada item é um registro
    .option("inferSchema", "true")
    .option("treatEmptyValuesAsNulls", "true")
    .load(fold)
)


In [0]:
display(purchase_items)

In [0]:
purchase_items.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save(F"{BRONZE_PATH}/purchase_orders")